In [13]:
#SHARED BLOCK

from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA256, SHA512, SHA1, HMAC
from Crypto.Signature import PKCS1_PSS
from Crypto.Signature import pkcs1_15
from sage.crypto.util import bin_to_ascii
import binascii
import secrets
import hashlib
from sage.crypto.block_cipher.miniaes import MiniAES

def binary_to_string(binary_string):
    # Split the binary string into chunks of 8 characters
    chunks = [binary_string[i:i+8] for i in range(0, len(binary_string), 8)]

    # Convert each chunk to its corresponding ASCII character
    ascii_chars = [chr(int(chunk, 2)) for chunk in chunks]

    # Join the ASCII characters to form the resulting string
    result = ''.join(ascii_chars)

    return result

def xor_bytes(lhs, rhs):
    return bytes(map(lambda x: x[0] ^^ x[1], zip(lhs, rhs)))

def prf(hash_func, secret: bytes, label: bytes, orig_seed: bytes, N):
    seed = label + orig_seed
    random_bytes = b''
    a = seed
    while len(random_bytes) < N:
        a = hmac(hash_func, secret, a)
        random_bytes += hmac(hash_func, secret, a + seed)
    return random_bytes[:N]

def hmac(hash_func, key: bytes, message: bytes):
    digest_size = hash_func().digest_size

    key_with_digest_size = key
    if len(key) < digest_size:
        key_with_digest_size += bytes(digest_size - len(key))
    elif len(key) > digest_size:
        key_with_digest_size = hash_func(key).digest()

    ipad = bytes([0x36 for _ in range(digest_size)])
    opad = bytes([0x5c for _ in range(digest_size)])

    key_with_ipad = xor_bytes(key_with_digest_size, ipad)
    key_with_opad = xor_bytes(key_with_digest_size, opad)

    return hash_func(key_with_opad + hash_func(key_with_ipad + message).digest()).digest()

def encrypt_long_message(plaintext, client_public_enc_key):
    chunk_size = 214
    cipher = PKCS1_OAEP.new(client_public_enc_key.publickey(), hashAlgo=SHA1)

    encrypted_chunks = []

    for i in range(0, len(plaintext), chunk_size):
        chunk = plaintext[i : i + chunk_size]
        print("[Encrypt] Size chunk_block = ", len(chunk))
        encrypted_ = cipher.encrypt(chunk)
        encrypted_chunks.append(encrypted_)

    return b"".join(encrypted_chunks)

def decrypt_long_message(encrypted_blocks, client_private_dec_key):
    chunk_size = 256
    cipher = PKCS1_OAEP.new(client_private_dec_key, hashAlgo=SHA1)

    decrypted_chunks = []

    for i in range(0, len(encrypted_blocks), chunk_size):
        chunk = encrypted_blocks[i : i + chunk_size]
        print("[Decrypt] Size chunk_block = ", len(chunk))
        assert len(chunk) == 256
        decrypted_ = cipher.decrypt(chunk)
        decrypted_chunks.append(decrypted_)

    return b"".join(decrypted_chunks)


def pad_input_string(input_string, block_size=16):
    pad_len = block_size - (len(input_string) % block_size)
    return input_string + chr(pad_len) * (pad_len)

def unpad_input_string(padded_string):
    pad_len = ord(padded_string[-1])
    return padded_string[:-pad_len]


def encrypt_mini_aes(plain_text, key):
    maes = MiniAES()
    bin = BinaryStrings()

    key_binary_string = bin(key.decode())

    print("Bits of Key Binary String = ", key_binary_string)
    print("Number of bits of Key Binary = ", len(key_binary_string))

    plain_text_string = pad_input_string(plain_text)
    P = bin.encoding(plain_text_string);

    C = maes(P, key_binary_string, algorithm="encrypt");

    return C

def decrypt_mini_aes(cipher, key):
    maes = MiniAES()
    bin = BinaryStrings()

    key_binary_string = bin(key.decode())

    print("[Decrypt] Bits of Key Binary String = ", key_binary_string)
    print("[Decrypt] Number of bits of Key Binary = ", len(key_binary_string))

    decrypted_text_padded = maes(cipher, key_binary_string, algorithm="decrypt")
    to_string = str(decrypted_text_padded)

    return unpad_input_string(to_string)

def add_extra_dollar(string_intput):
    string_intput = string_intput.strip()
    len_extra = 16
    for i in range(len_extra):
        string_intput += '$'
    return string_intput

def remove_extra_dollar(text):
    index = text.find('$')
    rem = len(text) - index
    text = text[:-rem]
    #print("Index = ", index)
    return text

In [19]:
# Server Block

class Server:
    def __init__(self):
        self.rsa_length_bits = 2048
        self.hash_func = SHA512

        # Enc and sign
        self.server_rsa_enc_key = RSA.generate(2048)
        self.server_rsa_sign_key = RSA.generate(2048)

        self.signature = None
        self.server_private_key = None

        self.client_rsa_enc_key_public = None
        self.client_rsa_sign_key_public = None

        self.num_rand_bytes = 32
        self.private_num_rand_bytes = 48
        self.premaster_secret = None
        self.master_secret = None
        self.client_private_random = None
        self.client_public_random = None

        self.client_write_mac_key = None
        self.server_write_mac_key = None
        self.client_write_key = None
        self.server_write_key = None

        self.block = None
        self.server_public_random = secrets.token_bytes(self.num_rand_bytes)
        self.server_private_random = secrets.token_bytes(self.private_num_rand_bytes)

    def generate_keys_server_side(self):
        # # Generate RSA keys on both sides for encryption and for signature
        server_rsa_enc_key = self.server_rsa_enc_key.e, self.server_rsa_enc_key.n
        server_rsa_sign_key = self.server_rsa_sign_key.e, self.server_rsa_sign_key.n

        print("Server RSA Public Enc Key = ", server_rsa_enc_key) # Public key, n
        print("Server RSA Public Sign Key = ", server_rsa_sign_key) # Public key, n

        return self.server_rsa_enc_key, self.server_rsa_sign_key

    def generate_public_random_32_bytes(self):
        # Generate 32 random bytes on both sides (client_public_random, server_public_random)
        print("Server Public 32 random bytes = ", self.server_public_random)
        print("Server Public [number of bytes -> 32]: ", len(self.server_public_random))

        return self.server_public_random

    def generate_private_random_48_bytes(self):
        # Generate 48 private random bytes on both sides (client_private_random, server_private_random)
        print("Server Private 48 random bytes = ", self.server_private_random)
        print("Server Private [number of bytes -> 48]: ", len(self.server_private_random))


    def sign_and_send(self, client_rsa_enc_key):
        # Sign server_private_random with RSA and send it from server to client with RSA-OAEP
        self.client_rsa_enc_key_public = client_rsa_enc_key

        hash = SHA512.new(self.server_private_random)
        server_signature = pkcs1_15.new(self.server_rsa_sign_key).sign(hash)

        print("[From client] Signature: ", server_signature)

        signed_server_signature = server_signature + self.server_private_random # Total = 512 + 48 = 560
        encrypted_signature_chunk = encrypt_long_message(signed_server_signature, client_rsa_enc_key)

        return encrypted_signature_chunk

    def receive_signature_from_client_and_verify(self, encrypted_signature_chunk, client_rsa_sign_key_public):
        # Decrypt the received message on server side and verify the signature
        self.client_rsa_sign_key_public = client_rsa_sign_key_public

        decrypted_signature = decrypt_long_message(encrypted_signature_chunk, self.server_rsa_enc_key)
        print("Server side the decrypted signature bytes: ", len(decrypted_signature))
        signature = decrypted_signature[:256]

        private_random_bytes = decrypted_signature[256:] # 48

        print("[From server] Decrypted private random = ", private_random_bytes)
        print("[From server] Decrypted signature = ", signature)

        hash = SHA512.new(private_random_bytes)
        verifier = pkcs1_15.new(client_rsa_sign_key_public)

        try:
            verifier.verify(hash, signature)

            print("", end="\n")
            print("The signature is authentic.")
            print("", end="\n")

            self.client_private_random = private_random_bytes

            print("Decrypted server private random = ", private_random_bytes)
            print("Client private random [number of bytes -> 48]: ", len(self.client_private_random))
            print("Server private random [number of bytes -> 48]: ", len(self.server_private_random))

        except:
            print("The signature is not authentic.")


    def combine_private_randoms(self):
        # Combine private randoms into premaster_secret = client_private_random + server_private_random, where +  is concatenation of bytes
        self.premaster_secret = self.client_private_random + self.server_private_random
        print("Pre master secret = ", self.premaster_secret)

        print("Pre master [number of bytes -> 96]: ", len(self.premaster_secret))


    def generate_master_secret(self):
        # Generate master secret with PRF
        self.concat_public_random = self.client_public_random + self.server_public_random

        self.master_secret = prf(hashlib.sha256, self.premaster_secret, b'master secret', self.concat_public_random, self.private_num_rand_bytes)

        print("Master secret = ", self.master_secret)

        return self.master_secret

    def get_client_public_random(self, client_public_random):
        self.client_public_random = client_public_random

    def generate_with_prf(self):
        # Generate the following keys with PRF
        total_bytes = 68 # 32 + 32 + 2 + 2 = 68

        seed = self.client_public_random + self.server_public_random
        self.block = prf(hashlib.sha256, self.master_secret, b'key expansion', seed, total_bytes)

        self.client_write_mac_key = self.block[0:32]
        self.server_write_mac_key = self.block[32:64]
        self.client_write_key = self.block[64:66]
        self.server_write_key = self.block[66:68]

        print("Number of bytes: ", len(self.server_write_key))

        print("Block = ", self.block)
        print("client_write_mac_key = ", self.client_write_mac_key)
        print("server_write_mac_key = ", self.server_write_mac_key)
        print("client_write_key = ", self.client_write_key)
        print("server_write_key = ", self.server_write_key)

        return self.client_write_mac_key, self.server_write_mac_key, self.client_write_key, self.server_write_key

    def send_message_from_server(self, message):
        # Send message from server to client with MiniAES(server_write_key) and HMAC(server_write_mac_key)
        padded_message = add_extra_dollar(message)
        key_binary = ''.join(format(byte, '08b') for byte in self.server_write_key)
        key_binary = key_binary.encode()
        encrypted = encrypt_mini_aes(padded_message, key_binary)

        key_bytes = self.server_write_mac_key
        authenticated = hmac(hashlib.sha256, key_bytes, padded_message.encode())

        print("[From server - Mini AES] Encrypted = ", encrypted)
        print("[From server - Mini AES] Authenticated = ", authenticated)

        return encrypted, authenticated

    def receive_and_verify_message(self, message, authentication_code):
        # Decrypt the message and verify the authentication code
        key_binary = ''.join(format(byte, '08b') for byte in self.client_write_key)
        key_binary = key_binary.encode()
        decrypted_padded = decrypt_mini_aes(message, key_binary)

        print("[From server - Mini AES] Decrypted = ", decrypted_padded)

        decrypted = remove_extra_dollar(binary_to_string(decrypted_padded))
        padded_message = add_extra_dollar(decrypted)
        
        print(end="\n")
        print(end="\n")
        print("Text:", decrypted)
        print(end="\n")

        print(end="\n")

        key_bytes = self.client_write_mac_key
        authenticated = hmac(hashlib.sha256, key_bytes, padded_message.encode())

        print("[From server - Mini AES] Authentication to check = ", authenticated)

        print("", end="\n")
        if authentication_code == authenticated:
            print("Verified sucessfully!")
        else:
            print("Failed to verify!")
        print("", end="\n")

        return decrypted


In [20]:
server = Server()
server_rsa_enc_key, server_rsa_sign_key = server.generate_keys_server_side()
server_public_random = server.generate_public_random_32_bytes()
server.generate_private_random_48_bytes()
encrypted_signature_chunk_server = server.sign_and_send(server_rsa_enc_key.publickey())
print("Client chunk: ", encrypted_signature_chunk_server)
print("", end="\n")
print("", end="\n")
server.receive_signature_from_client_and_verify(encrypted_signature_chunk_server, server_rsa_sign_key.publickey())

Server RSA Public Enc Key =  (65537, 20761853438516929529082210986597505834036722659996075371299333492093360503407657958552751390292037223184397539652966293850222572075923604293661362528589044377531625504294144116271323180093811500530705391077362277586251880516387500307572534725296644088109120886758176295268353218297545833368660464374665292518517595587490531000630294167414172842963308672326716655251206387661635373832697796751851456407507133258352999780121666411683074739113201259639526904538314526857531664057081169863801409759477705390187603512930418691458279927354264558730768760158509859452979941690974512775363912772151821682789309932058989098781)
Server RSA Public Sign Key =  (65537, 232192609147448515806668399823681329000435223436880805624460824526084670556451434496306923909094444866401841369757791796276841134461832045894810019712729039464771540889066430777202030666306171289015131464681430408807179603860470004326541550561868852007940829910362199118373060728151590653923436102212040398

In [21]:
# Client Block

class Client:
    def __init__(self):
        self.rsa_length_bits = 2048
        self.hash_func = SHA512

        # Enc and sign
        self.client_rsa_enc_key = RSA.generate(2048)
        self.client_rsa_sign_key = RSA.generate(2048)

        self.signature = None
        self.client_private_key = None

        self.server_rsa_enc_key_public = None
        self.server_rsa_sign_key_public = None

        self.num_rand_bytes = 32
        self.private_num_rand_bytes = 48
        self.premaster_secret = None
        self.master_secret = None
        self.server_private_random = None
        self.server_public_random = None

        self.client_write_mac_key = None
        self.server_write_mac_key = None
        self.client_write_key = None
        self.server_write_key = None

        self.block = None
        self.client_public_random = secrets.token_bytes(self.num_rand_bytes)
        self.client_private_random = secrets.token_bytes(self.private_num_rand_bytes)

    def generate_keys_client_side(self):
        # Generate RSA keys on both sides for encryption and for signature
        client_rsa_enc_key = self.client_rsa_enc_key.e, self.client_rsa_enc_key.n
        client_rsa_sign_key = self.client_rsa_sign_key.e, self.client_rsa_sign_key.n

        print("Client RSA Public Enc Key = ", client_rsa_enc_key) # Public key, n
        print("Client RSA Public Sign Key = ", client_rsa_sign_key) # Public key, n

        return self.client_rsa_enc_key, self.client_rsa_sign_key

    def generate_public_random_32_bytes(self):
        # Generate 32 random bytes on both sides (client_public_random, server_public_random)
        print("Client Public 32 random bytes = ", self.client_public_random)
        print("Client Public [number of bytes -> 32]: ", len(self.client_public_random))
        return self.client_public_random

    def generate_private_random_48_bytes(self):
        # Generate 48 private random bytes on both sides (client_private_random, server_private_random)
        print("Client Private 48 random bytes = ", self.client_private_random)
        print("Client Private [number of bytes -> 48]: ", len(self.client_private_random))

    def sign_and_send(self, server_rsa_enc_key_public):
        # Sign client_private_random with RSA and send it from client to server with RSA-OAEP
        self.server_rsa_enc_key_public = server_rsa_enc_key_public

        hash = SHA512.new(self.client_private_random)
        client_signature = pkcs1_15.new(self.client_rsa_sign_key).sign(hash) # 256 bytes

        print("[From client] Signature: ", client_signature)

        signed_client_signature = client_signature + self.client_private_random # 256 + 48 bytes = 304 bytes
        encrypted_signature_chunk = encrypt_long_message(signed_client_signature, server_rsa_enc_key_public)

        return encrypted_signature_chunk

    def receive_signature_from_server_and_verify(self, encrypted_signature_chunk, server_rsa_sign_key_public):
        # Decrypt the received message on client side and verify the signature
        self.server_rsa_sign_key_public = server_rsa_sign_key_public
        print("Process decrypted -- orignal encrypted_signature_chunk: ", len(encrypted_signature_chunk)) # 512 bytes
        decrypted_signature = decrypt_long_message(encrypted_signature_chunk, self.client_rsa_enc_key)
        print("Client side the decrypted signature bytes: ", len(decrypted_signature)) # 304 bytes

        signature = decrypted_signature[:256]

        private_random_bytes = decrypted_signature[256:]

        print("[From client] Decrypted private random = ", private_random_bytes)
        print("[From client] Decrypted signature = ", signature)

        hash = SHA512.new(private_random_bytes) # 
        verifier = pkcs1_15.new(server_rsa_sign_key_public) # <- server_rsa_enc_key_public

        try:
            verifier.verify(hash, signature)
            print("", end="\n")
            print("The signature is authentic.")
            print("", end="\n")
            self.server_private_random = private_random_bytes
            print("Decrypted server private random = ", self.server_private_random)
            print("Server Private random [number of bytes -> 48]: ", len(self.server_private_random))
            print("Client random [number of bytes -> 48]: ", len(self.client_private_random))
        except:
            print("The signature is not authentic.")

    def get_server_public_random(self, server_public_random):
        self.server_public_random = server_public_random

    def generate_with_prf(self):
        # Generate the following keys with PRF
        total_bytes = 68

        seed = self.client_public_random + self.server_public_random
        self.block = prf(hashlib.sha256, self.master_secret, b'key expansion', seed, total_bytes)

        self.client_write_mac_key = self.block[0:32]
        self.server_write_mac_key = self.block[32:64]
        self.client_write_key = self.block[64:66]
        self.server_write_key = self.block[66:68]

        print("Number of bytes: ", len(self.client_write_key))

        print("Block = ", self.block)
        print("client_write_mac_key = ", self.client_write_mac_key)
        print("server_write_mac_key = ", self.server_write_mac_key)
        print("client_write_key = ", self.client_write_key)
        print("server_write_key = ", self.server_write_key)

        return self.client_write_mac_key, self.server_write_mac_key, self.client_write_key, self.server_write_key

    def get_master_secret(self, master_secret):
        # Generate master secret with PRF
        self.master_secret = master_secret

    def send_message_from_client(self, message):
        # Send message from client to server with MiniAES(client_write_key) and HMAC(client_write_mac_key)
        padded_message = add_extra_dollar(message)
        key_binary = ''.join(format(byte, '08b') for byte in self.client_write_key)
        key_binary = key_binary.encode()
        encrypted = encrypt_mini_aes(padded_message, key_binary)

        key_bytes = self.client_write_mac_key
        authenticated = hmac(hashlib.sha256, key_bytes, padded_message.encode())

        print("[From client - Mini AES] Encrypted = ", encrypted)
        print("[From client - Mini AES] Authenticated = ", authenticated)

        return encrypted, authenticated

    def receive_and_verify_message(self, message, authentication_code):
        # Decrypt the message and verify the authentication code
        key_binary = ''.join(format(byte, '08b') for byte in self.server_write_key)
        key_binary = key_binary.encode()
        decrypted_padded = decrypt_mini_aes(message, key_binary)

        print("[From client - Mini AES] Decrypted = ", decrypted_padded)

        decrypted = remove_extra_dollar(binary_to_string(decrypted_padded))
        padded_message = add_extra_dollar(decrypted)
        
        print(end="\n")
        print(end="\n")
        print("Text:", decrypted)
        print(end="\n")

        print(end="\n")
        key_bytes = self.server_write_mac_key
        authenticated = hmac(hashlib.sha256, key_bytes, padded_message.encode())

        print("[From client - Mini AES] Authentication to check = ", authenticated)

        print("", end="\n")
        if authentication_code == authenticated:
            print("Verified sucessfully!")
        else:
            print("Failed to verify!")
        print("", end="\n")

        return decrypted

In [22]:
server = Server()
client = Client()

server_rsa_enc_key, server_rsa_sign_key = server.generate_keys_server_side()
client_rsa_enc_key, client_rsa_sign_key = client.generate_keys_client_side()

server_public_random = server.generate_public_random_32_bytes()
client_public_random = client.generate_public_random_32_bytes()

server.generate_private_random_48_bytes()
client.generate_private_random_48_bytes()

encrypted_signature_chunk_server = server.sign_and_send(client_rsa_enc_key.publickey())
encrypted_signature_chunk_client = client.sign_and_send(server_rsa_enc_key.publickey()) # server_rsa_enc_key

print("Client chunk: ", encrypted_signature_chunk_server)
print("Server chunk: ", encrypted_signature_chunk_client)

print("", end="\n")
print("", end="\n")

server.receive_signature_from_client_and_verify(encrypted_signature_chunk_client, client_rsa_sign_key.publickey())
client.receive_signature_from_server_and_verify(encrypted_signature_chunk_server, server_rsa_sign_key.publickey()) # server_rsa_sign_key

Server RSA Public Enc Key =  (65537, 26580509397643415115083515594648010732006666174364708734990098227723537004978775469483000931355701556499776455704333186252575363720076584558719552330357940785505663590647183796887460591422288717492491021788127208856002060086705725298729149688432091736439134253838187735735007578046753323854375097201399080016535421926212437374874245920940434902668725072147671111022076994486643927814699290349318961293939911348300659403181778317521231003156519367094108011441057482162848991346013033479329168726356936846106046241202717698564670705409437973649743282873485380468225135931062555600993536856396062257681767793669490580161)
Server RSA Public Sign Key =  (65537, 243899748457875078548897700423425362096668038345862448076903170340578391447334124274347735196557740845602696011970878813388331467411718430898672882226088763558462729637465736336243473351438942364892647999655020898745252088145663439137110454670395654916955479155039278414861047473041721660573609189715683618

In [23]:
print("", end="\n")
print("", end="\n")

server.get_client_public_random(client_public_random)
client.get_server_public_random(server_public_random)

server.combine_private_randoms()
master_secret = server.generate_master_secret()
client.get_master_secret(master_secret)

print("", end="\n")
print("", end="\n")

server.generate_with_prf()
message_from_server = "Hi, Client!"
encrypted_from_server, authenticated_from_server = server.send_message_from_server(message_from_server)

print("", end="\n")

client.generate_with_prf()
message_from_client = "Hi, Server!"
encrypted_from_client, authenticated_from_client = client.send_message_from_client(message_from_client)

print("", end="\n")
print("", end="\n")

server.receive_and_verify_message(encrypted_from_client, authenticated_from_client)
client.receive_and_verify_message(encrypted_from_server, authenticated_from_server)

i = 3
assert(i == 3)



Pre master secret =  b'\x92AsH\x87\x1ev\x19\xd0\xdf\x0c\x01l\x1f\xc8A\xf87\xa1\x96K\x80\x01\x12\xf7\xccOCw\xf3\xc9\xe7\xeb,\xc3\xb6\xde)Y\x08\x06\\f\xd0\xae\xe3\x1c\xfe\x9c\xc8\xc6\xea\xcb\xfb"\xe7"\n\n\x902\xa1p0ja\xc9L\x19\xa3D\xfcrK\x95\x823;\xd7\x19w\xf8\x19\x11\xe0\xbd\x93ruBme\xab\xe6~\t'
Pre master [number of bytes -> 96]:  96
Master secret =  b"Q~p9\x1b\x99^\xbe\xd3em\x1d\x1e\xff\x9b\xd9\x03\xdc\x043\xc7\xf4\x84(\xc1k7\xa3FO\x9b\xda\xea\xe2_]'ZYtY\xdeK\xd8)\xe4\xaeU"


Number of bytes:  2
Block =  b"\xda\xbf5=\x16 LO\xef \xb4\\\x9b\xb8\x8fq\x97\x86\x81C\xf5\xe7\xca\xe2'i\xc7\xa6\xebNI\xaeZ\x06q\xc8\xfe\xf7\x0f6\xdb\xf6\x14\xb4F\xc0\xd5H[\x99\x97\x99\x85\x9d\x86\xa9\xe6\xbd\x93\xd5(\xdc\x8b~{\xa8\xf1\n"
client_write_mac_key =  b"\xda\xbf5=\x16 LO\xef \xb4\\\x9b\xb8\x8fq\x97\x86\x81C\xf5\xe7\xca\xe2'i\xc7\xa6\xebNI\xae"
server_write_mac_key =  b'Z\x06q\xc8\xfe\xf7\x0f6\xdb\xf6\x14\xb4F\xc0\xd5H[\x99\x97\x99\x85\x9d\x86\xa9\xe6\xbd\x93\xd5(\xdc\x8b~'
client_write_key =  b'{\xa8'

[From client - Mini AES] Decrypted =  010010000110100100101100001000000100001101101100011010010110010101101110011101000010000100100100001001000010010000100100001001000010010000100100001001000010010000100100001001000010010000100100001001000010010


Text: Hi, Client!


[From client - Mini AES] Authentication to check =  b'\xfav\xd5\xed\x98\xe0\xd2\x8e\x12\xd6E(&\xf5\x01\r^\x0eZ\xa1\xd7\x81\x107\xa7\xb6\xbd\xb76\xad^\xac'

Verified sucessfully!

